In [ ]:
try:
  from bs4 import BeautifulSoup
except:
  !pip install bs4
  from bs4 import BeautifulSoup

try:
  import requests
except:
  !pip install requests
  import requests

try:
  import pandas as pd
except:
  !pip install pandas
  import pandas as pd

try:
  import string
except:
  !pip install string
  import string

try:
  import os
except:
  !pip install os
  import os

try:
  import pathlib
except:
  !pip install pathlib
  import pathlib

try:
  import nltk
except:
  !pip install nltk
  import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
input = pd.read_excel('/content/drive/MyDrive/Projects/BlackCoffer/Input.xlsx')
output = pd.DataFrame(columns = ['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE','SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT','SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])
input.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [ ]:
output.columns

Index(['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [ ]:
try:
  os.makedirs('/content/drive/MyDrive/Projects/BlackCoffer/articles/')
except:
  pass
for i, url in enumerate(input.URL):
  r = requests.get(url = url)
  soup = BeautifulSoup(r.content, 'html5lib')
  header = soup.title.text.replace(' - Blackcoffer Insights', "")
  article = soup.find('div', attrs={'class':'td-post-content'})
  try:
    article = ''.join(str(a.text) for a in article(['p', 'ul', 'ol']))
  except:
    article = 'No content found.'
  with open(f'/content/drive/MyDrive/Projects/BlackCoffer/articles/{input.URL_ID[i]}.txt', 'w+') as txtfile:
    txtfile.write(f'{header}\n')
    txtfile.write(article)

In [ ]:
s = pathlib.Path('/content/drive/MyDrive/Projects/BlackCoffer/StopWords/')
stopwords = ""
for file in s.iterdir():
  with open(file, 'r', encoding = "ISO-8859-1") as f:
    stopwords += f.read()
sw_list = stopwords.split('\n')

In [ ]:
try:
  os.makedirs('/content/drive/MyDrive/Projects/BlackCoffer/articles/clean/')
except:
  pass

p = pathlib.Path('/content/drive/MyDrive/Projects/BlackCoffer/articles')
for i, file in enumerate(p.glob('*.txt')):
  with open(file, 'r') as text:
    tokens = text.read().split()
    text = [w for w in tokens if not (w in sw_list)]
    clean_text = " ".join(text)
    clean_text = clean_text.translate(str.maketrans(string.punctuation.replace('.', ""), " " * (len(string.punctuation)-1)))   #to remove punctuations except period(.)
    with open(f'/content/drive/MyDrive/Projects/BlackCoffer/articles/clean/{input.URL_ID[i]}_cleaned.txt', 'w+') as txtfile:
      txtfile.write(clean_text)

In [ ]:
def vowel_count(s):
    vowels = frozenset('aeiouAEIOU')
    return sum(1 for c in s if c in vowels)

pos_dict = open('/content/drive/MyDrive/Projects/BlackCoffer/MasterDictionary/positive-words.txt').read()
neg_dict = open('/content/drive/MyDrive/Projects/BlackCoffer/MasterDictionary/negative-words.txt', encoding = "ISO-8859-1").read()
clean = pathlib.Path('/content/drive/MyDrive/Projects/BlackCoffer/articles/clean')

for i, file in enumerate(clean.iterdir()):
  with open(file, 'r') as f:
    pos_score = 0
    neg_score = 0
    text = f.read()
    tokens = nltk.word_tokenize(text)
    for word in tokens:
      if word in pos_dict:
        pos_score += 1
      elif word in neg_dict:
        neg_score += 1

    total_words = len([word.replace('.','') for word in tokens])
    text_noPunc = " ".join(word for word in tokens)
    polarity = (pos_score - neg_score)/((pos_score + neg_score) + 0.000001)
    subjectivity = (pos_score + neg_score)/(total_words + 0.000001)
    number_of_sentences = text.count('.')
    avg_sent_len = total_words/number_of_sentences
    syllables = sum([vowel_count(word) for word in tokens if not word.endswith('es') or word.endswith('ed')])/total_words
    complex_words = len([word for word in text.split() if  vowel_count(word)> 2])
    percent_complex = (complex_words / total_words) * 100
    fog_index = 0.4 * (avg_sent_len + percent_complex)
    prsl_pronouns = sum([1 if word.lower() in ["i", "we", "my", "ours", "us"] else 0  for word in tokens])
    output.loc[i] = [input.loc[i]['URL_ID'], input.loc[i]['URL'],pos_score, neg_score, polarity, subjectivity, avg_sent_len, percent_complex , fog_index, avg_sent_len, complex_words, total_words, syllables, prsl_pronouns, len(text_noPunc)/total_words]

In [ ]:
output.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,146,72,0.339450,0.289125,9.544304,34.748011,17.716926,9.544304,262,754,1.785146,5,6.350133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,195,100,0.322034,0.304438,11.962963,49.432405,24.558147,11.962963,479,969,2.182663,3,7.434469
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,118,66,0.282609,0.247312,13.285714,54.166667,26.980952,13.285714,403,744,2.471774,0,8.258065
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,122,85,0.178744,0.295292,13.745098,53.495007,26.896042,13.745098,375,701,2.490728,1,8.263909
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,81,26,0.514019,0.250000,10.700000,44.158879,21.943551,10.700000,189,428,2.203271,1,7.686916


In [ ]:
output.to_csv('Output.csv')
input.to_csv('Input.csv')